<a href="https://colab.research.google.com/github/seagulley/GloVe/blob/master/Understanding_chain_rule_annotated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding the Chain Rule (with Code)

Welcome! This notebook turns the **chain rule** into something you can see, code, and reason about.  
We’ll move from **intuition → formal math → implementation → experiments & plots**.

## Learning objectives
By the end, you will be able to:
- State and use the scalar **chain rule** for composite functions.
- Interpret the **vector/Jacobian** form via computational graphs.
- Implement a tiny gradient routine that applies the chain rule.
- Verify gradients numerically and (optionally) with autodiff.
- Read plots that illustrate gradient flow and sensitivity.

> **How to use this notebook:** Each section begins with a short explanation.  
> Some cells are marked **(Advanced)** and include more formal derivations. These are optional “extra‑mile” materials.


## Notebook Roadmap

1. **Setup & helpers** — imports, utilities
2. **Symbolic derivatives (optional)** — use algebra to check the chain rule
3. **From scratch implementation** — tiny gradient code that applies the chain rule
4. **Numeric checks** — finite differences to validate gradients
5. **Autodiff sanity check (optional)** — compare against PyTorch autograd if present
6. **Visualizations** — see how composition affects sensitivities
7. **Mini‑exercises** — quick prompts to reinforce ideas


## The Chain Rule — Formal Statements (Advanced)

**Scalar → scalar (single variable):** If $ y = f(g(x)) $ and both $f$ and $g$ are differentiable at the appropriate points, then
$
\frac{dy}{dx} \;=\; f'(g(x)) \cdot g'(x).
$

**Scalar → scalar (multi‑stage composition):** For $ y = f(g(h(x))) $,
$
\frac{dy}{dx} \;=\; f'(g(h(x)))\cdot g'(h(x))\cdot h'(x).
$

**Vector form (Jacobian chain rule):** Let $ \mathbf{u} = g(\mathbf{x}) \in \mathbb{R}^m $, $ \mathbf{y} = f(\mathbf{u}) \in \mathbb{R}^p $. Then for differentiable $f,g$,
$
J_{f\circ g}(\mathbf{x}) \;=\; J_f(\mathbf{u})\; J_g(\mathbf{x}) \quad\text{with}\quad \mathbf{u}=g(\mathbf{x}).
$
Equivalently, for a scalar output $y$ and vector input $\mathbf{x}$,
$
\nabla_{\mathbf{x}}\, y \;=\; J_g(\mathbf{x})^\top\, \nabla_{\mathbf{u}}\, y.
$

**Computational graph view:** If a node $z$ depends on parents $\{a_i\}$, then a small change $d a_i$ induces
$
dz \;=\; \sum_i \frac{\partial z}{\partial a_i}\, d a_i.
$
Backpropagation applies this repeatedly from outputs to inputs.


# Multivariate function and its derivative

_(Annotated)_

### Autodiff / Backprop Sanity Check (Optional)

If PyTorch (or another autodiff library) is available, we check our results against `autograd`.

> **What happens here?** We build tensors that require gradients and let the library compute `dy/dx` automatically for comparison.

In [1]:
import torch
from torch.autograd import Variable
from torch.autograd import Function
import torch.nn.functional as F
import numpy as np

x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)
f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]+2.0*x[2])**4 + 10.0*(x[0]-x[3])**4

print("Vector variable x:",x.data)

print("Function f at x:",f.data)

# compute gradient of f at x
g = torch.autograd.grad(f,x)

print("Gradient of f at x:",g[0].data)

Vector variable x: tensor([ 3., -1.,  0.,  1.])
Function f at x: tensor(215.)
Gradient of f at x: tensor([ 306., -144.,  -18., -310.])


### Let's verify with math formula.

$f(x_1,x_2,x_3,x_4) = (x_1+10x_2)^2 + 5(x_3-x_4)^2 + (x_2+2x_3)^4 + 10(x_1-x_4)^4$

Let's evaluate these partial derivatives at

$x_1 = 3, x_2 = -1, x_3 = 0, x_4 = 1$

$\frac{\partial f}{\partial x_1} = 2(x_1+10x_2)+40(x_1-x_4)^3 = 2(3-10)+40(3-1)^3 = -14+320 = 306$

$\frac{\partial f}{\partial x_2} = 20(x_1+10x_2)+4(x_2+2x_3)^3 = 20(3-10)+4(-1)^3 = -140-4 = -144$

$\frac{\partial f}{\partial x_3} = 10(x_3-x_4)+8(x_2+2x_3)^3 = 10(-1)+8(-1)^3 = -10-8 = -18$

$\frac{\partial f}{\partial x_4} = -10(x_3-x_4)-40(x_1-x_4)^3 = -10(-1)-40(3-2)^3 = 10-320 = -310$

So, we have the gradient

$\nabla f(3,-1,0,1) = \left[ 306,-144,-18,-310 \right]$


_(Annotated)_

# Numerical Derivative
### For any function we can also compute its numerical derivative, which is an approximation. The formula for computing numerical derivatives often uses Taylor series approximation from Calculus:
$\frac{\partial f}{\partial x_1} = \frac{f(x_1+\delta x_1,x_2,x_3,...)-f(x_1-\delta x_1,x_2,x_3,...)}{2\delta x_1},$

$\frac{\partial f}{\partial x_2} = \frac{f(x_1,x_2+\delta x_2,x_3,...)-f(x_1,x_2-\delta x_2,x_3,...)}{2\delta x_2},$

$...,$

### and so on for each variable $x_i$. As a result, comoutation of numerical derivative is extremely inefficient for a function with a large number of variables. But it is often used as a check to see if your autograd code is working correctly, or not.


_(Annotated)_

> **What happens here?** We build tensors that require gradients and let the library compute `dy/dx` automatically for comparison.

In [ ]:
x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)

def compute_f(x):
  f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]+2.0*x[2])**4 + 10.0*(x[0]-x[3])**4
  return f

f = compute_f(x)

print("Vector variable x:",x.data)

print("Function f at x:",f.data)

# compute gradient of f at x
g = torch.autograd.grad(f,x)

print("Exact gradient of f at x:",g[0].data)

num_g = torch.zeros(4)
h=1e-3
eye = torch.eye(4) # 4-by-4 identity matrix
for i in range(4):
  num_g[i] = compute_f(x+h*eye[i,:]) - compute_f(x-h*eye[i,:])

num_g = num_g/(2.0*h)

print("Numerical gradient of f at x:",num_g.data)

Vector variable x: tensor([ 3., -1.,  0.,  1.])
Function f at x: tensor(215.)
Exact gradient of f at x: tensor([ 306., -144.,  -18., -310.])
Numerical gradient of f at x: tensor([ 305.9769, -143.9972,  -18.0054, -309.9976])


# Gradient descent

_(Annotated)_

> **What happens here?** We build tensors that require gradients and let the library compute `dy/dx` automatically for comparison.

In [12]:
x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)

steplength = 1e-3 # for gradient descent
for i in range(5):
  # function
  f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]+2.0*x[2])**4 + 10.0*(x[0]-x[3])**4
  # compute grdaient
  g = torch.autograd.grad(f,x)
  print(g[0].shape)
  print("x:", x)
  print("g:", g[0])
  # adjust variable
  x = x - steplength*g[0]
  if i%100==0:
    print("Current variable:",x.detach().numpy(),"Current function value:", f.item())


torch.Size([4])
x: tensor([ 3., -1.,  0.,  1.], requires_grad=True)
g: tensor([ 306., -144.,  -18., -310.])
Current variable: [ 2.694 -0.856  0.018  1.31 ] Current function value: 215.0
torch.Size([4])
x: tensor([ 2.6940, -0.8560,  0.0180,  1.3100], grad_fn=<SubBackward0>)
g: tensor([  94.3077, -119.5255,  -17.3309,  -93.1197])
torch.Size([4])
x: tensor([ 2.5997, -0.7365,  0.0353,  1.4031], grad_fn=<SubBackward0>)
g: tensor([ 58.9994, -96.4817, -16.0392, -54.8516])
torch.Size([4])
x: tensor([ 2.5407, -0.6400,  0.0514,  1.4580], grad_fn=<SubBackward0>)
g: tensor([ 43.0520, -77.8050, -15.3066, -36.7044])
torch.Size([4])
x: tensor([ 2.4976, -0.5622,  0.0667,  1.4947], grad_fn=<SubBackward0>)
g: tensor([ 34.1085, -62.8002, -14.9109, -26.0769])


# Gradient descent using PyTorch's optmization

_(Annotated)_

> **What happens here?** We build tensors that require gradients and let the library compute `dy/dx` automatically for comparison.

In [ ]:
x = Variable(torch.tensor([3.0,-1.0,0.0,1.0]),requires_grad=True)

optimizer = torch.optim.SGD([x], lr=1e-3, momentum=0.9) # create an optimizer that will do gradient descent optimization

for i in range(100):
  f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]+2.0*x[2])**4 + 10.0*(x[0]-x[3])**4
  optimizer.zero_grad()
  f.backward()
  optimizer.step()
  if i%10==0:
    print("Current variable:",x.detach().numpy(),"Current function value:", f.item())


Current variable: [ 2.694 -0.856  0.018  1.31 ] Current function value: 215.0
Current variable: [ 1.549712   -0.2964205   0.73650396  1.6820586 ] Current function value: 11.441824913024902
Current variable: [ 1.408362   -0.00583249  0.57971793  1.0532441 ] Current function value: 6.6437225341796875
Current variable: [ 0.6763986  -0.03913701  0.24533094  1.1681743 ] Current function value: 5.709615707397461
Current variable: [ 0.5996511  -0.13040473  0.5036107   0.53555894] Current function value: 1.093517541885376
Current variable: [ 0.59016085 -0.07009609  0.31826916  0.27739534] Current function value: 0.27947038412094116
Current variable: [ 0.5269876  -0.03624894  0.12591662  0.26216415] Current function value: 0.16900770366191864
Current variable: [ 0.45544916 -0.04479674  0.13050896  0.22580846] Current function value: 0.08700942993164062
Current variable: [ 0.399531   -0.04679754  0.18029977  0.18939461] Current function value: 0.034884385764598846
Current variable: [ 0.35905084 

# Chain rule of derivative

_(Annotated)_

> **What happens here?** We build tensors that require gradients and let the library compute `dy/dx` automatically for comparison.

In [ ]:
z = Variable(torch.tensor([1.0,-1.0]),requires_grad=True)

print("Variable z:",z.data)

def compute_x(z):
  x = torch.zeros(4)
  x[0] = z[0] - z[1]
  x[1] = z[0]**2
  x[2] = z[1]**2
  x[3] = z[0]**2+z[0]*z[1]
  return x

x = compute_x(z)
print("function x:",x.data)

def compute_f(x):
  f = (x[0]+10.0*x[1])**2 + 5.0*(x[2]-x[3])**2 + (x[1]+2.0*x[2])**4 + 10.0*(x[0]-x[3])**4
  return f

f = compute_f(x)
print("function f:",f.item())
print("")
#
# Let's compute gradient of f with respect to x
g_x = torch.autograd.grad(f,x,retain_graph=True,create_graph=True)
print("Gradient of f with respect x:",g_x[0].data)
print("")
# Now compute Jacobian of x with respect to z and multiply with g_x to use chain rule
g_z = torch.autograd.grad(x,z,g_x,retain_graph=True)

# But PyTorch can compute derivative of f with respect to z directly - this is the amazing capability!
g = torch.autograd.grad(f,z)

print("Gradient of f with respec to z by two-step calculation:",g_z[0].data)
print("Gradient of f with respec to z by one-step calculation:",g[0].data)

Variable z: tensor([ 1., -1.])
function x: tensor([2., 1., 1., 0.])
function f: 390.0

Gradient of f with respect x: tensor([ 344.,  348.,  226., -330.])

Gradient of f with respec to z by two-step calculation: tensor([  710., -1126.])
Gradient of f with respec to z by one-step calculation: tensor([  710., -1126.])


## Let's verify the result using math formula

$z_1 = 1, z_2 = -1$

$x_1 = z_1 - z_2 = 2, x_2 = z_1^2 = 1, x_3 = z_2^2 = 1, x_4 = z_1^2+z_1z_2 = 0$

## Let's compute partial derivative of $f$ with respect to $x_i$:

$\frac{\partial f}{\partial x_1} = 2(x_1+10x_2)+40(x_1-x_4)^3 = 2(2+10)+40(2-0)^3 = 24+320 = 344$

$\frac{\partial f}{\partial x_2} = 20(x_1+10x_2)+4(x_2+2x_3)^3 = 20(2+10)+4(1+2)^3 = 348$

$\frac{\partial f}{\partial x_3} = 10(x_3-x_4)+8(x_2+2x_3)^3 = 10(1)+8(3)^3 = 10+216 = 226$

$\frac{\partial f}{\partial x_4} = -10(x_3-x_4)-40(x_1-x_4)^3 = -10(1)-40(2)^3 = -10-320 = -330$

## Let's now compute partial derivative of $x_i$ with respect to $z_1$:

$\frac{\partial x_1}{\partial z_1} = 1$

$\frac{\partial x_2}{\partial z_1} = 2z_1 = 2(1) = 2$

$\frac{\partial x_3}{\partial z_1} = 0$

$\frac{\partial x_4}{\partial z_1} = 2z_1 + z_2 = 2(1)-1 = 1$

## Let's also compute partial derivative of $x_i$ with respect to $z_2$:

$\frac{\partial x_1}{\partial z_2} = -1$

$\frac{\partial x_2}{\partial z_2} = 0$

$\frac{\partial x_3}{\partial z_2} = 2z_2 = -2$

$\frac{\partial x_4}{\partial z_2} = z_1 = 1$

## Now we can apply the chain rule of derivative:

$\frac{\partial f}{\partial z_1} = \sum_i \frac{\partial f}{\partial x_i}\frac{\partial x_i}{\partial z_1} = 344(1) + 348(2) + 226(0) -330(1) = 710$

$\frac{\partial f}{\partial z_2} = \sum_i \frac{\partial f}{\partial x_i}\frac{\partial x_i}{\partial z_2} = 344(-1) + 348(0) + 226(-2) -330(1) = -1126$

## Using Jacobian notation, chain rule can be written as follows:

\begin{equation}
\nabla_z f =
    \begin{bmatrix}
        \frac{\partial f}{\partial z_1}\\
        \frac{\partial f}{\partial z_2}
    \end{bmatrix} =
    (J_z^x) (\nabla_x f) =
    \begin{bmatrix}
        \frac{\partial x_1}{\partial z_1} & \frac{\partial x_2}{\partial z_1} & \frac{\partial x_3}{\partial z_1} & \frac{\partial x_4}{\partial z_1}\\
        \frac{\partial x_1}{\partial z_2} & \frac{\partial x_2}{\partial z_2} & \frac{\partial x_3}{\partial z_2} & \frac{\partial x_4}{\partial z_2}
    \end{bmatrix}
    \begin{bmatrix}
        \frac{\partial f}{\partial x_1}\\
        \frac{\partial f}{\partial x_2}\\
        \frac{\partial f}{\partial x_3}\\
        \frac{\partial f}{\partial x_4}
    \end{bmatrix} =
    \begin{bmatrix}
        1 & 2 & 0 & 1\\
        -1 & 0 & -2 & 1
    \end{bmatrix}    
    \begin{bmatrix}
        344\\
        348\\
        226\\
        -330
    \end{bmatrix} =
    \begin{bmatrix}
        710\\
        -1126
    \end{bmatrix}
\end{equation}


_(Annotated)_

# Optimization by gradient descent: Assume $f$ is a loss function
## We are minimizing $f$ by adjusting $z_1$ and $z_2$.

_(Annotated)_

> **What happens here?** We build tensors that require gradients and let the library compute `dy/dx` automatically for comparison.

In [ ]:
steplength = 1e-3 # for gradient descent
for i in range(1000):
  # function
  f = compute_f(compute_x(z))
  # Compute gradient of f with respect to z directly
  # PyTorch takes care of chain rule of derivatives
  g = torch.autograd.grad(f,z)
  # adjust variable
  z = z - steplength*g[0]
  if i%100==0:
    print("Current variable (z) value:",z.detach().numpy(),"Current function value:", f.item())

Current variable (z) value: [0.28999996 0.12600005] Current function value: 390.0
Current variable (z) value: [0.10041686 0.16472499] Current function value: 0.002078988589346409
Current variable (z) value: [0.0918402  0.16432461] Current function value: 0.0010676763486117125
Current variable (z) value: [0.08976527 0.1616178 ] Current function value: 0.0009482738096266985
Current variable (z) value: [0.08847897 0.15887022] Current function value: 0.0008560288697481155
Current variable (z) value: [0.08736441 0.15630378] Current function value: 0.0007776079582981765
Current variable (z) value: [0.08633856 0.15392138] Current function value: 0.000710221123881638
Current variable (z) value: [0.08538311 0.15170398] Current function value: 0.0006518377340398729
Current variable (z) value: [0.08448897 0.14963281] Current function value: 0.0006008768104948103
Current variable (z) value: [0.08364932 0.14769198] Current function value: 0.000556100916583091


# And of course optimization using PyTorch's gradient descent optimization

_(Annotated)_

> **What happens here?** We build tensors that require gradients and let the library compute `dy/dx` automatically for comparison.

In [ ]:
z = Variable(torch.tensor([1.0,-1.0]),requires_grad=True)

optimizer = torch.optim.SGD([z], lr=1e-4, momentum=0.9) # create an optimizer that will do gradient descent optimization

for i in range(100):
  f = compute_f(compute_x(z))
  optimizer.zero_grad()
  f.backward()
  optimizer.step()
  if i%10==0:
    print("Current variable (z) value:",z.detach().numpy(),"Current function value:", f.item())


Current variable (z) value: [ 0.929      -0.88740003] Current function value: 390.0
Current variable (z) value: [-0.19928932  0.4960087 ] Current function value: 0.9813486337661743
Current variable (z) value: [-0.44388917  0.7940718 ] Current function value: 24.753379821777344
Current variable (z) value: [-0.15692     0.49109557] Current function value: 2.4734973907470703
Current variable (z) value: [0.00094466 0.29169223] Current function value: 0.23995020985603333
Current variable (z) value: [0.06613181 0.20713682] Current function value: 0.02455221861600876
Current variable (z) value: [0.09276582 0.17456606] Current function value: 0.0017178322887048125
Current variable (z) value: [0.10217317 0.16280487] Current function value: 0.0023682680912315845
Current variable (z) value: [0.10384288 0.1590711 ] Current function value: 0.003257445292547345
Current variable (z) value: [0.10240351 0.1583022 ] Current function value: 0.002950004767626524


### Example / Exploration

An exploratory cell that supports the narrative above.

> **What happens here?** Supporting computations for the surrounding explanation.

## Mini‑Exercises

1. Let $ y = \sin(3x^2+1) $. Compute $dy/dx$ by hand and then verify numerically.
2. For $ \mathbf{x}\in\mathbb{R}^2 $, $ u = W\mathbf{x} + \mathbf{b} $ (affine), $ y = \sigma(u_1 u_2) $. Derive $
\nabla_{\mathbf{x}} y $ using Jacobians.
3. Replace the inner function with a nonlinearity (e.g., $g(x)=	\tanh(x)$ and plot how the gradient $dy/dx$ changes across inputs.
4. (Advanced) Implement reverse‑mode accumulation on a small computational graph and print local partials as you backpropagate.
